
## 線形化する

非線形なままだと扱いづらくて困るので，似ている線形システムに置き換えてしまいます．ここでは $θ$ が小さいと仮定して線形化します．これは，振子の傾きがそんなに大きくないということを意味します．

* 三角関数はゼロの周りの接線により近似をします．このとき，$\sin θ ≈ θ, \cos θ ≈ 1$ と近似できます．
* また，$θ$ が小さいと仮定したので $\dot{θ}^2 ≈ 0$ と近似できます．（なぜ？）

よって運動方程式は近似的に次のようになります．

$$
(M + m) \ddot{x} + m l \ddot{θ} = f \\\\
ml \ddot{x} + ml^2 \ddot{θ} + mgl θ = 0
$$


## 状態方程式を求める

さらにこれを状態空間モデルとして解釈するために，状態ベクトル $X$ を考えます．状態は $x, \dot{x}, θ, \dot{θ}$ の 4 つなので
$$
X = \begin{bmatrix} x \\ \dot{x} \\ θ \\ \dot{θ} \end{bmatrix}
$$
とします．

$X$ の微分を $X$ の線形変換として書き表すために，$\ddot{x}$ と $\ddot{θ}$ を $X$ と $f$ の線型結合で表現する必要があります．これも制御というより式の計算の話なので詳細は省略しますが，実行すると以下のようになります:

$$
\begin{align}
  \ddot{x} &= \frac{mg}{M} θ + \frac{1}{M} f \\\\
  \ddot{θ} &= -\frac{(m + M)g}{M} θ - \frac{1}{M} f
\end{align}
$$

これにより，状態方程式は次のようになります．

$$
\begin{align}
  A &= \begin{bmatrix} 
    0 & 1 & 0 & 0 \\ 
    0 & 0 & \frac{mg}{M} & 0 \\ 
    0 & 0 & 0 & 1 \\ 
    0 & 0 & -\frac{(m + M)g}{M} & 0 
  \end{bmatrix} \\\\
  B &= \begin{bmatrix} 0 \\ \frac{1}{M} \\ 0 \\ -\frac{1}{M} \end{bmatrix} \\\\
  \ddot{X} &= A X + B f
\end{align}
$$

制御出力は $y = (x, θ)^T$ なので，出力方程式は次のようになります．

$$
\begin{align}
  C &= \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \end{bmatrix} \\\\
  D &= \begin{bmatrix} 0 \\ 0 \end{bmatrix} \\\\
  y &= C X + D f
\end{align}
$$



In [ ]:
# そのままだと日本語が文字化けするので，日本語を表示するための設定をする

# 必要なパッケージをインポート
using Plots

# エンコーディングをutf8に変更
ENV["GKS_ENCODING"] = "utf8"

# grの環境のフォントファミリーを日本語のものに
gr(fontfamily="IPAMincho")

plot(
    [],
    [],
    xlabel = "台車の位置",
    ylabel = "振子の角度",
    title = "倒立振子のシミュレーション",
    # lw = 1,
)

In [ ]:
# 必要なパッケージをインポート
using Plots

# エンコーディングをutf8に変更
ENV["GKS_ENCODING"] = "utf8"

# grの環境のフォントファミリーを日本語のものに
gr(fontfamily="IPAMincho")

# 初期状態の設定
initial_state = State(0.0, 0.0, 0.0, 0.0) # 台車の位置、振子の角度、台車の速度、振子の角速度

# 制御入力の設定
u = 0.1 # 任意の制御入力を設定

# シミュレーションのパラメータ
dt = 0.01 # シミュレーションの時間刻み
simulation_time = 10.0 # シミュレーションの時間

# シミュレーション用の配列を初期化
num_steps = Int(simulation_time / dt)
states = [initial_state]

# シミュレーションループ
for i in 1:num_steps
    # 現在の状態を取得
    current_state = states[end]

    # 振子が倒れてしまっていたら，シミュレーションを終了する
    if abs(current_state.θ) >= π / 2
        println("Pendulum fell beyond threshold at time ", i * dt)
        break
    end

    # 次の状態を計算
    next_state = cartpole(current_state, u)

    # 次の状態を配列に追加
    push!(states, next_state)
end

# 台車の位置と振子の角度をプロット
plot(
    [state.x for state in states], # 台車の位置
    [state.θ for state in states], # 振子の角度
    xlabel = "台車の位置",
    ylabel = "振子の角度",
    label = ["Cart Position" "Pendulum Angle"],
    title = "Cart-Pole Simulation",
    lw = 2,
)